### ספריות

In [1]:
import os
import sys
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### העלת משתנים להרצת הקוד

In [3]:
path = os.getcwd()

software_root_folder = os.path.dirname(path)

software_folder_location = r'{}\current'.format(software_root_folder)

df_inputs_outputs = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(software_folder_location))

forecast_version_folder_location=df_inputs_outputs['location'][1]

output_location=df_inputs_outputs['location'][2]

sys.path.append(software_root_folder)

In [11]:
df_inputs_outputs

,discription,location
0,מיקום תוכנת תחזית בסיס,C:\Users\dpere\Documents\JTMT\forecast\create_...
1,מיקום פלט לפי גירסא,C:\Users\dpere\Documents\JTMT\forecast_by_vers...
2,מיקום פלט,C:\Users\dpere\Documents\JTMT\forecast_by_vers...
3,שכבה חדשה,False
4,מיקום שכבה חדשה,NaN


### פונקציות

In [4]:
def find_files_with_pattern(directory, pattern):
    """
    Find files in a directory that match a certain pattern.
    
    Args:
    - directory (str): The directory path.
    - pattern (str): The pattern to search for in file names.
    
    Returns:
    - List of file paths matching the pattern.
    """
    files = []
    for root, _, filenames in os.walk(directory):
        for filename in filenames:
            if pattern in filename:
                files.append(os.path.join(root, filename))
    return files

### העלת שכבות רלוונטים

In [5]:
matching_files = find_files_with_pattern(output_location, "forecast_2020")

In [6]:
prototypes_df=pd.read_excel(r"{}\background_files\age_des_types.xlsx".format(software_folder_location))

In [7]:
data_df = pd.read_excel(matching_files[0])

In [8]:
filename = os.path.basename(matching_files[0])

# Remove the file extension if present
filename = os.path.splitext(filename)[0]

In [ ]:
sectors=gpd.read_file(r'{}\background_files\TAZ_V4_230518_Published.shp'.format(software_folder_location))

In [ ]:
data_df=sectors.merge(data_df,left_on='Taz_num',right_on='TAZ',how='right')

In [ ]:
data_df['sector_for_age']=data_df['sector']

In [ ]:
data_df.loc[
    (data_df['zonetype'] == 'Judea and Samaria') &
    (data_df['sector'] != 'U_Orthodox'),'sector_for_age'
]='hitnachlut'


In [ ]:
lst_muni=['מעלה אדומים',
 'גבעת זאב',
 'הר אדר',
 '0',
 'מעלה אפרים',
 'מגילות ים המלח\r\n',
 'קרני שומרון',
 'אלפי מנשה',
 'בית אריה-עופרים',
 'ערבות הירדן\r\n',
 'אריאל']

In [ ]:
data_df.loc[data_df['Muni_Heb'].isin(lst_muni),'sector_for_age'
]='Jewish'


In [ ]:
data_df.loc[
    (data_df['sector'] == 'arabs_behined_seperation_wall') |
    (data_df['sector'] == 'Arab'),'sector_for_age'
]='Arab'


In [ ]:
data_df=data_df.loc[
    (data_df['sector'] != 'Palestinian')
]


In [ ]:
col=[ 'age0_4',
 'age5_9',
 'age10_14',
 'age15_19',
 'age20_24',
 'age25_29',
 'age30_34',
 'age35_39',
 'age40_44',
 'age45_49',
 'age50_54',
 'age55_59',
 'age60_64',
 'age65_69',
 'age70_74',
 'age75up']

In [ ]:
# Convert counts to percentages
data_df_pre = data_df[col].apply(lambda x: x / x.sum(), axis=1)

In [ ]:
data_df=data_df.join(data_df_pre,lsuffix='', rsuffix='_prec')

In [ ]:
col_name_prototype=['pop_0','pop_5',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up']

In [ ]:
col_name_row=['age0_4_prec',
 'age5_9_prec',
 'age10_14_prec',
 'age15_19_prec',
 'age20_24_prec',
 'age25_29_prec',
 'age30_34_prec',
 'age35_39_prec',
 'age40_44_prec',
 'age45_49_prec',
 'age50_54_prec',
 'age55_59_prec',
 'age60_64_prec',
 'age65_69_prec',
 'age70_74_prec',
 'age75up_prec']

In [ ]:
ls_sector=['U_Orthodox', 'Jewish', 'hitnachlut', 'Arab']

In [ ]:
for s in ls_sector:
    locals()['data_df_{}'.format(s)]=data_df.loc[data_df['sector_for_age']=='{}'.format(s)]
    locals()['prototypes_df_{}'.format(s)]=prototypes_df.loc[prototypes_df['sector']=='{}'.format(s)]

In [ ]:
for s in ls_sector:
    for index, row in locals()['data_df_{}'.format(s)].iterrows():
        # Step 3: Calculate similarity scores
        similarity_scores = []
        for _, prototype_row in locals()['prototypes_df_{}'.format(s)].iterrows():
            prototype_age_distributions = prototype_row[col_name_prototype].values 
            data_age_distributions = row[col_name_row].values         
            similarity_score = np.linalg.norm(prototype_age_distributions - data_age_distributions)
            similarity_scores.append(similarity_score)

        # Step 4: Determine closest prototype
        closest_index = np.argmin(similarity_scores)
        closest_prototype = locals()['prototypes_df_{}'.format(s)].iloc[closest_index]['classification_name']
        

        # Assign closest_prototype to the corresponding row in data_df or perform any desired action

        locals()['data_df_{}'.format(s)].at[index,'classification_name'] = closest_prototype
        

In [ ]:
data_df=pd.concat([data_df_U_Orthodox,data_df_Jewish,data_df_hitnachlut,data_df_Arab],axis=0)

In [ ]:
# data_df[['Taz_num','sector_for_age','classification_name']].to_excel(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\התפלגות גילים\backround_files\{}_age_dis_type.xlsx'.format(filename),index=False)
data_df[['Taz_num','sector_for_age','classification_name']].to_excel(r'{}\Intermediates\{}_age_dis_type.xlsx'.format(software_folder_location,filename),index=False)